# VGG 16

### 라이브러리 import

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 모델 구현

In [ ]:
class VGG_16(nn.Module):
    def __init__(self):
        super(VGG_16,self).__init__()
        self.convnet = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 224 -> 112

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 112 -> 56

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 56 -> 28
            
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 28 -> 14

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            # 14 -> 7

            nn.Flatten(), # Flatten은 layer로 안쳐준다.
            nn.Dropout(.5),
            nn.Linear(in_features=512*7*7, out_features=4096),
            nn.ReLU(),
            nn.Dropout(.5),
            nn.Linear(in_features=4096, out_features=4096),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=1000)
        )
    def forward(self, x):
        return self.convnet(x)
    

In [ ]:
model = VGG_16()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=model.parameters(), lr=1e-2, momentum=.9, weight_decay=5e-4)

### 학습 함수

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader) # 사실 없어도 됨 마지막에 가독성을 좋게하기위해 사용
    model.train()   # torch안의 module안에 train이라는 메소드가 있다.
    
    for batch, data in enumerate(dataloader):
        image, label = data

        # Compute prediction error 
        pred = model(image) # model.forward(image)
        # softmax = nn.Softmax(dim=1)
        # pred_probab = softmax(pred)
        loss = loss_fn(pred, label)

        # backpropagation
        loss.backward() # 쓸모없는 노드를 지운다. # 매개변수를 안정해주면 랜덤으로 잡고 경사하강을 한다.
        optimizer.step() #learning rate 만큼 내려간다.
        optimizer.zero_grad() #초기화해주는데 해주는 이유는 이전 스텝을 기억하고있으면 다음 스텝에 영향을 줄 수 있기때문에 초기화한다.

        if not(batch % 100):
            loss, current = loss.item(), (batch + 1) * len(image)
            print(f"loss : {loss:>7f}   [{current:>5d}/{size:>5d}]")

### 테스트 함수

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() #평가를 해주겠다 명시
    test_loss, correct = 0, 0
    
    with torch.no_grad(): #no_grad -> 최적화를 수행하지 않겠다.
        for image, label in dataloader:
            pred = model(image)
            # softmax = nn.Softmax(dim=1)
            # pred_probab = softmax(pred)
            test_loss += loss_fn(pred, label).item() #.item() -> loss_fn에 있는 데이터값을 불러준다.
            correct += (pred.argmax(1) == label).type(torch.float).sum().item()
                            #argmax --> 예측값중에 가장 큰 값을 뽑아오는 것
    
    test_loss /= num_batches
    correct /=size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

### 실행

In [ ]:
epochs = 30
for i in range(epochs):
    print(f"Epoch {i + 1}\n------------------------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done")